# 大模型提示词工程

## 1. 提示词基础

提示词工程也称为“In-Context Prompting”，其实就是我们使用大模型的最最基本的方式：对话。我们向大模型提出要求，大模型理解我们的意图，给出答案。
在这个过程中，我们通过改变话术就可以影响大模型输出答案的准确性，这个不断与大模型对话不断校准获得我们想要的答案的过程，就是提示词工程。

下面我们来看一个简单的案例，首先我们定义访问函数如下：

In [29]:
import os
from dotenv import load_dotenv

## for MacOS users
filePath = os.path.abspath(os.path.expanduser(os.path.expandvars("~/.zshrc")))
load_dotenv(filePath)

Python-dotenv could not parse statement starting at line 7
Python-dotenv could not parse statement starting at line 8
Python-dotenv could not parse statement starting at line 10
Python-dotenv could not parse statement starting at line 11
Python-dotenv could not parse statement starting at line 16


True

## 制作一个LLM查询客户端

In [30]:
import sys
from langchain_community.llms import Tongyi

def Q(question):
    llm = Tongyi(streaming=True)
    llm.model_name = 'qwen-turbo'
    for chunk in llm.stream(question):
        print(chunk, end="", flush=True)

In [31]:
# Q("请续写儿歌，第一句是：太阳当空照")

In [32]:
Q("海湾战争的起因是什么？")

海湾战争是以1990年8月2日伊拉克入侵科威特开始的，是二战后参战国家最多的一场局部战争。其直接起因是伊拉克和科威特之间关于边界纠纷和石油问题的争执。伊拉克认为科威特的石油资源丰富，对其领土有野心；而科威特则寻求国际社会的支持来维护其独立和主权。在联合国安理会未能达成和平解决协议的情况下，伊拉克悍然出兵占领了科威特，引发了国际社会的强烈谴责和反对。以美国为首的多国联盟随后组成联军，对伊拉克进行军事干预，最终迫使伊拉克撤军，恢复了科威特的主权。这场战争也展示了现代高科技战争的特点，包括大规模使用空中打击和电子战。

In [33]:
Q("请把上面的话翻译成英文")

Please translate the above statement into English.

### 1.1 实现简单多轮对话

上述介绍的零提示词法并不涉及记忆能力，也就是说两个问句之间没有内在联系。

但是，当我们在使用大模型应用时通常会感觉对话很自然，这是因为真实大模型应用是多轮对话模式，大模型是有记忆的。

如何实现这一点呢？我们考虑大模型对话的基础仍然是简单提示词对话。因此，如果我们把上一轮的“对话内容”巧妙加入到当前对话中，我们就应该能实现和大模型的多轮对话。

当然，这个对话的历史长度取决于大模型可输入的tokens总量。

接下来，我们尝试考虑上下文，实现一个多轮对话。请尝试如下操作：

In [9]:
import json
from langchain_community.llms import Tongyi

class MyAgent:
    def __init__(self, mem_len=10, model_name = 'qwen-max'):
        self.memory = []
        self.men_len = mem_len
        self.model_name = model_name

    def QA(self, question, new_context=False):
        llm = Tongyi(streaming=True)
        llm.model_name = self.model_name

        if(new_context==True):
            self.memory = []
        prompt = {"question":question, "history":self.memory}
        
        ans=""
        for chunk in llm.stream(json.dumps(prompt , ensure_ascii=False)):
            ans += chunk
            print(chunk, end="", flush=True)
        self.memory.append((question, ans))

        if(len(self.memory)>self.men_len):
            self.memory.pop(0)

        self.last_ans = ans
        # return ans

llm = MyAgent()

In [8]:
llm.QA("阿里云ECS能做什么？")

阿里云Elastic Compute Service（ECS）是一种可弹性伸缩的云计算服务，提供安全、可靠、弹性的计算能力。使用阿里云ECS，用户可以轻松构建、部署和管理各种类型的应用，满足不同业务场景的需求。具体来说，阿里云ECS能够帮助用户实现以下功能和服务：

1. **部署应用**：ECS实例作为虚拟服务器，支持广泛的操作系统（如Windows、Linux等），用户可以在其上安装、配置和运行各类应用程序，包括Web服务器（如Apache、Nginx）、数据库（如MySQL、PostgreSQL）、中间件、编程语言环境（如Java、Python、Node.js）、CRM系统、ERP系统、电子商务平台、游戏服务器等。

2. **网站托管**：无论是个人博客、中小企业官网、大型电商网站还是高并发访问的新闻门户，都可以在ECS上搭建和运行。用户可以选择适合的实例规格（CPU、内存、存储等资源组合）和带宽方案，确保网站的稳定访问和快速响应。

3. **数据分析与处理**：对于大数据分析、机器学习、人工智能等需要大量计算资源的任务，用户可以创建高性能计算型或GPU型ECS实例，搭载相关软件（如Hadoop、Spark、TensorFlow等）进行高效的数据处理和模型训练。

4. **云端开发测试环境**：ECS可以为开发人员快速创建隔离的开发、测试、 staging环境，方便进行代码编写、编译、调试、集成测试等工作。实例资源可根据需求随时调整，有效降低研发成本，提高开发效率。

5. **云上科学研究**：科研机构或个人研究人员可以利用ECS的强大计算能力进行生物信息学分析、气候模拟、分子动力学计算、天文数据处理等科学计算任务。

6. **容器化应用部署**：ECS与阿里云容器服务ACK（ACK）深度集成，支持在ECS实例上高效运行Docker容器，实现微服务架构、持续集成/持续部署（CI/CD）等现代化应用交付方式。

7. **云端游戏服务器**：游戏开发者可以使用ECS部署游戏逻辑服务器、数据库服务器、登录服务器等组件，借助阿里云的全球数据中心网络，为玩家提供低延迟、高可用的游戏体验。

8. **云上教育培训**：教育机构可以利用ECS搭建在线教育平台、远程实验环境、编程教学实验室等，为学生提供灵活便捷的学习资源。

9. **存储与备份**：结合阿里云

'阿里云Elastic Compute Service（ECS）是一种可弹性伸缩的云计算服务，提供安全、可靠、弹性的计算能力。使用阿里云ECS，用户可以轻松构建、部署和管理各种类型的应用，满足不同业务场景的需求。具体来说，阿里云ECS能够帮助用户实现以下功能和服务：\n\n1. **部署应用**：ECS实例作为虚拟服务器，支持广泛的操作系统（如Windows、Linux等），用户可以在其上安装、配置和运行各类应用程序，包括Web服务器（如Apache、Nginx）、数据库（如MySQL、PostgreSQL）、中间件、编程语言环境（如Java、Python、Node.js）、CRM系统、ERP系统、电子商务平台、游戏服务器等。\n\n2. **网站托管**：无论是个人博客、中小企业官网、大型电商网站还是高并发访问的新闻门户，都可以在ECS上搭建和运行。用户可以选择适合的实例规格（CPU、内存、存储等资源组合）和带宽方案，确保网站的稳定访问和快速响应。\n\n3. **数据分析与处理**：对于大数据分析、机器学习、人工智能等需要大量计算资源的任务，用户可以创建高性能计算型或GPU型ECS实例，搭载相关软件（如Hadoop、Spark、TensorFlow等）进行高效的数据处理和模型训练。\n\n4. **云端开发测试环境**：ECS可以为开发人员快速创建隔离的开发、测试、 staging环境，方便进行代码编写、编译、调试、集成测试等工作。实例资源可根据需求随时调整，有效降低研发成本，提高开发效率。\n\n5. **云上科学研究**：科研机构或个人研究人员可以利用ECS的强大计算能力进行生物信息学分析、气候模拟、分子动力学计算、天文数据处理等科学计算任务。\n\n6. **容器化应用部署**：ECS与阿里云容器服务ACK（ACK）深度集成，支持在ECS实例上高效运行Docker容器，实现微服务架构、持续集成/持续部署（CI/CD）等现代化应用交付方式。\n\n7. **云端游戏服务器**：游戏开发者可以使用ECS部署游戏逻辑服务器、数据库服务器、登录服务器等组件，借助阿里云的全球数据中心网络，为玩家提供低延迟、高可用的游戏体验。\n\n8. **云上教育培训**：教育机构可以利用ECS搭建在线教育平台、远程实验环境、编程教学实验室等，为学生提供灵活便捷的学习资源。\n\

In [4]:
llm.QA("ECS 能部署应用吗？")

是的，ECS 可以部署应用。您可以将各种类型的应用程序部署到阿里云ECS实例上，包括但不限于Web应用、大数据应用、机器学习模型、游戏服务器、软件开发环境等。您可以直接在ECS实例上安装所需的应用服务器、数据库和其他软件，然后配置和管理您的应用。此外，通过容器服务（如Kubernetes或Docker），ECS还可以支持更加现代化和自动化应用的部署和管理。

In [5]:
llm.QA("把ECS特性的第二项翻译成英文")

The second feature of ECS is "Hosting websites and applications".

In [6]:
llm.memory

[('阿里云ECS能做什么？',
  '阿里云ECS（Elastic Compute Service）是阿里云提供的弹性计算服务，它允许用户在云端获取和管理计算资源。通过使用ECS，您可以快速部署和运行各种应用程序、网站或服务，而无需投资和维护底层硬件。以下是阿里云ECS可以做的事情：\n\n1. **托管网站和应用**：您可以部署和运行个人、企业或电商网站，以及各种Web应用程序，如内容管理系统（CMS）、论坛、博客等。\n\n2. **大数据处理**：ECS可提供弹性的计算能力，支持大规模数据处理和分析任务，如Hadoop、Spark等分布式计算框架。\n\n3. **机器学习与人工智能**：ECS可以作为机器学习模型训练和推理的平台，支持TensorFlow、PyTorch等AI框架。\n\n4. **游戏服务器**：游戏开发者可以使用ECS来托管游戏服务器，提供稳定的游戏体验。\n\n5. **软件开发和测试**：ECS可以快速创建和销毁开发环境，用于软件开发、调试和测试。\n\n6. **媒体转码**：阿里云ECS可以配合其他服务，如媒体转换服务，进行视频编码和转码工作。\n\n7. **云计算服务**：作为基础设施即服务（IaaS）的一部分，ECS可以作为其他阿里云服务的基础，如数据库服务、CDN、负载均衡等。\n\n8. **在线教育**：支持实时音视频互动，构建在线课堂和教育平台。\n\n9. **容器服务**：ECS可以与Kubernetes、Docker等容器技术结合，实现更高效的应用部署和管理。\n\n10. **高可用和容灾**：通过阿里云的负载均衡和多区域部署，实现业务的高可用性和灾难恢复。\n\n总的来说，阿里云ECS是一个功能强大的计算平台，可以满足各种企业和个人的计算需求，从简单的网站托管到复杂的云原生应用部署，都能提供灵活、可扩展的解决方案。'),
 ('ECS 能部署应用吗？',
  '是的，ECS 可以部署应用。您可以将各种类型的应用程序部署到阿里云ECS实例上，包括但不限于Web应用、大数据应用、机器学习模型、游戏服务器、软件开发环境等。您可以直接在ECS实例上安装所需的应用服务器、数据库和其他软件，然后配置和管理您的应用。此外，通过容器服务（如Kubernetes或Docker），ECS还可以支持更加现代化和自动化应用的部署

## 指令性提示词
### 工作邮件

In [16]:
question = "请用中文撰写一封邮件给团队成员，通知他们下周将有一次团队建设活动，活动的主题是团队协作，请大家积极参加"

llm.QA(question,new_context=True)

In [17]:
question = '''请用中文撰写一封邮件给团队成员，通知他们下周将有一次团队建设活动，活动的主题是团队协作，请大家积极参加。\
       已知信息，需要打车前往，请大家组团，地点在奥林匹克公园。时间定在周五下午，大概四个小时'''

llm.QA(question,new_context=True)


### 代码生成与改写

In [18]:
question = '''将以帮我生成一段冒泡排序的python代码'''

llm.QA(question,new_context=True)


当然可以，以下是一段使用Python实现的冒泡排序算法代码：

```python
def bubble_sort(arr):
    """
    冒泡排序函数，对输入的列表arr进行升序排序

    参数:
        arr (list): 待排序的列表

    返回:
        list: 排序后的列表
    """

    n = len(arr)
    for i in range(n):
        # 每轮遍历都会将当前未排序部分的最大值"冒泡"到末尾
        for j in range(0, n - i - 1):
            if arr[j] > arr[j + 1]:
                # 交换相邻元素，确保较小的元素在前
                arr[j], arr[j + 1] = arr[j + 1], arr[j]

    return arr

# 示例：使用冒泡排序对一个整数列表进行排序
unsorted_list = [64, 34, 25, 12, 22, 11, 90]
sorted_list = bubble_sort(unsorted_list)
print("Sorted List:", sorted_list)
```

这段代码定义了一个名为`bubble_sort`的函数，用于执行冒泡排序。函数接受一个待排序的列表作为参数，通过两层循环逐次比较并交换相邻元素的位置，确保每一轮遍历后，当前未排序部分的最大值被正确地放置在其最终位置（即当前已排序部分的末尾）。当所有元素都经过这样的“冒泡”过程后，列表即完成排序。最后，我们提供了一个示例，对一个包含整数的列表进行冒泡排序，并打印出排序后的结果。

#### 代码改写

In [13]:
question = '''将以下Python代码转换为等效的Ruby代码。同时请注意保留所有注释，以帮助Ruby程序员理解代码的功能。
#Calculate the factorial of a number
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n - 1)'''


llm.QA(question,new_context=True)


```ruby
# Calculate the factorial of a number
def factorial(n)
  # Base case: factorial of 0 is 1
  return 1 if n == 0

  # Recursive case: factorial of n is n times the factorial of (n - 1)
  n * factorial(n - 1)
end
```

已将给定的Python代码转换为等效的Ruby代码，并保留了原有的注释以帮助Ruby程序员理解代码功能。

# 课上实验

#### 实验1 公文总结

In [19]:
question = '''
       《生成式人工智能服务管理暂行办法》

第一章 总　则

第一条 为了促进生成式人工智能健康发展和规范应用，维护国家安全和社会公共利益，保护公民、法人和其他组织的合法权益，根据《中华人民共和国网络安全法》、《中华人民共和国数据安全法》、《中华人民共和国个人信息保护法》、《中华人民共和国科学技术进步法》等法律、行政法规，制定本办法。

第二条 利用生成式人工智能技术向中华人民共和国境内公众提供生成文本、图片、音频、视频等内容的服务（以下称生成式人工智能服务），适用本办法。

国家对利用生成式人工智能服务从事新闻出版、影视制作、文艺创作等活动另有规定的，从其规定。

行业组织、企业、教育和科研机构、公共文化机构、有关专业机构等研发、应用生成式人工智能技术，未向境内公众提供生成式人工智能服务的，不适用本办法的规定。

第三条 国家坚持发展和安全并重、促进创新和依法治理相结合的原则，采取有效措施鼓励生成式人工智能创新发展，对生成式人工智能服务实行包容审慎和分类分级监管。

第四条 提供和使用生成式人工智能服务，应当遵守法律、行政法规，尊重社会公德和伦理道德，遵守以下规定：

（一）坚持社会主义核心价值观，不得生成煽动颠覆国家政权、推翻社会主义制度，危害国家安全和利益、损害国家形象，煽动分裂国家、破坏国家统一和社会稳定，宣扬恐怖主义、极端主义，宣扬民族仇恨、民族歧视，暴力、淫秽色情，以及虚假有害信息等法律、行政法规禁止的内容；

（二）在算法设计、训练数据选择、模型生成和优化、提供服务等过程中，采取有效措施防止产生民族、信仰、国别、地域、性别、年龄、职业、健康等歧视；

（三）尊重知识产权、商业道德，保守商业秘密，不得利用算法、数据、平台等优势，实施垄断和不正当竞争行为；

（四）尊重他人合法权益，不得危害他人身心健康，不得侵害他人肖像权、名誉权、荣誉权、隐私权和个人信息权益；'''


llm.QA(question,new_context=True)

编写提示词，让大模型用30个字总结上面的文档

In [23]:
instruction='''[这里写要求]'''

message = instruction +"\n"+ question

llm.QA(message,new_context=True)

#### 实验2 图表解读和预测能力

以下是一个虚构的服装企业的月销售和生产数据。

数据说明
- **日期**: 2023年1月至2023年12月
- **销售数量**: 每月销售的服装单位数（千件）
- **生产数量**: 每月生产的服装单位数（千件）

数据集
| 月份  | 销售数量 (千件) | 生产数量 (千件) |
|-------|---------------|----------------|
| 1月  | 50            | 60             |
| 2月  | 45            | 55             |
| 3月  | 55            | 65             |
| 4月  | 60            | 75             |
| 5月  | 70            | 80             |
| 6月  | 75            | 85             |
| 7月  | 80            | 90             |
| 8月  | 85            | 95             |
| 9月  | 90            | 100            |
| 10月 | 95            | 105            |
| 11月 | 100           | 110            |
| 12月 | 105           | 100            |

请基于上述的数据预测未来3个月的销售情况


In [ ]:
question = '''【请写入提示词】'''


llm.QA(question,new_context=True)

#### 实验3 图表分析

请尝试输入如下要求，观察大模型的执行结果，你也可以自己提供一些数据来做测试。

数据集：社交媒体广告活动数据

|广告系列编号|展示次数|点击次数|转化次数|总花费（$）|
|---|---|---|---|---|
|A|10,000|500|50|1,000|
|B|15,000|750|55|1,500|
|C|8,000|640|48|800|
|D|20,000|1,200|60|2,400|

问题：
- 哪一广告系列的点击率（点击次数/展示次数）最高？
- 哪一广告系列的转化率（转化次数/点击次数）最高？
- 根据总花费计算每个广告系列的平均转化成本（总花费/转化次数）。

In [ ]:
question = '''【请写入提示词】'''


llm.QA(question,new_context=True)